## Import libs

In [211]:
import requests as re
from time import sleep
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import sqlite3
from tqdm.notebook import tqdm

PATH_TO_DB = '../Board-Game-Data/board_games.sqlite'

In [3]:
users = pd.read_csv('../Board-Games/data/full/bgg_users_full.csv')
users

,link,nickname,country,state_cuty
0,/user/ me,me,England,England</div>
1,/user/00296,00296,England,England<br/>US Armed Forces - Europe</div>
2,/user/08bhattrell,08bhattrell,England,England</div>
3,/user/08pm,08pm,England,England<br/>Michigan</div>
4,/user/100pcBlade,100pcBlade,England,England<br/>Crawley<br/>West Sussex</div>
...,...,...,...,...
1665199,/user/UndergroundMan,UndergroundMan,Tanzania,Tanzania<br/>Unknown Village<br/>Arusha Region...
1665200,/user/vortexturd,vortexturd,Tanzania,Tanzania</div>
1665201,/user/woodwardmw,woodwardmw,Tanzania,Tanzania<br/>Dar es Salaam<br/>Dar es Salaam</...
1665202,/user/X3N0,X3N0,Tanzania,Tanzania<br/>Unknown Village<br/>Arusha Region...


In [4]:
ratings = pd.read_feather('../Board-Games/data/full/bgg_ratings_full.feather')
ratings

,nickname,title,boardgame_id,rating,num_of_plays,comment,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,last_modified
0,DiVo1975,Anachrony,185343,9.0,2,None,0,1,0,0,0,0,0,0,2022-11-06 14:02:30
1,DiVo1975,Anachrony: Infinity Box,278292,10.0,2,None,1,0,0,0,0,0,0,0,2022-01-06 06:21:59
2,DiVo1975,Ark Nova,342942,10.0,25,None,1,0,0,0,0,0,0,0,2022-09-05 01:21:16
3,DiVo1975,Arkham Horror: The Card Game,205637,9.0,0,None,1,0,0,0,0,0,0,0,2022-11-06 14:02:55
4,DiVo1975,Blood Rage,170216,10.0,1,None,1,0,0,0,0,0,0,0,2022-11-06 14:03:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20186643,zinnwen,Taj Mahal,475,7.0,0,None,0,0,0,0,0,0,0,0,2009-05-25 03:54:26
20186644,zinnwen,Through the Ages: A New Story of Civilization,182028,9.0,0,None,0,0,0,0,0,0,0,0,2017-09-01 11:38:04
20186645,zinnwen,Ticket to Ride,9209,6.0,0,None,0,0,0,0,0,0,0,0,2009-05-25 03:54:31
20186646,zinnwen,Tigris & Euphrates,42,8.0,0,None,0,0,0,0,0,0,0,0,2009-05-25 03:53:41


In [13]:
boardgames = pd.read_feather('../Board-Games/data/full/bgg_boardgames.feather')
boardgames

,boardgame_id,title,year_published,minplayers,maxplayers,minplaytime,maxplaytime,age,users_rated,average_rating,...,graphic_designers,subdomains,implementations,suggested_numplayers,podcast_episodes,comments,marketplace_history,marketplace_listings,thumbnail_link,image_link
0,5848,Rockit to Number 1,2002,2,8,40,40,0,10,5.825,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,King4alifetime|Would love to see an update on ...,,,https://cf.geekdo-images.com/pNnHBeoBxmZ2GajhG...,https://cf.geekdo-images.com/pNnHBeoBxmZ2GajhG...
1,259048,Star Wars: X-Wing (Second Edition) – RZ-2 A-Wi...,2018,2,2,30,45,14,39,8.0865,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,cammo35|For Outter Rim||Cullensdad|Own 4 copie...,,,https://cf.geekdo-images.com/N0aX_CzHdNSzBEREy...,https://cf.geekdo-images.com/N0aX_CzHdNSzBEREy...
2,122324,Wer bin ich? Das Promi-Rate-Spiel,2003,2,6,40,40,12,0,0,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,https://cf.geekdo-images.com/7Lc3uvftR0sI9TGtj...,https://cf.geekdo-images.com/7Lc3uvftR0sI9TGtj...
3,239964,Slam Burger,2017,4,8,0,0,6,7,6.92871,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,bert8freudenkinder|fun!||fs1973|LeiriaCON copy,unix_saledate|1521510090||saletime|85885||pric...,,https://cf.geekdo-images.com/wIdJWjYScBDpUbspX...,https://cf.geekdo-images.com/wIdJWjYScBDpUbspX...
4,227582,Cthulhu: Rise of the Cults,2017,1,4,90,120,14,109,6.57199,...,,,,1||Best~0|Recommended~0|Not Recommended~1|||2|...,,adamdynris|Kickstarter janvier 2018||AgentQ|I ...,unix_saledate|1516883073||saletime|4553993||pr...,,https://cf.geekdo-images.com/9ORSJup7z0llS8zQN...,https://cf.geekdo-images.com/9ORSJup7z0llS8zQN...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134735,373860,Tricky Rails,2022,4,4,45,60,12,5,8.2,...,,,,4+||,,BankofDracula|DJ Kenel has been mucking around...,,,https://cf.geekdo-images.com/SVt_sNGFFOI40hCq2...,https://cf.geekdo-images.com/SVt_sNGFFOI40hCq2...
134736,114857,Dougram Simulation Manual,1982,2,2,30,30,0,1,6,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,AMI_SOLGER|Fast action tactical combat game.\n...,,,https://cf.geekdo-images.com/bjdWaEjC0XleaCDWF...,https://cf.geekdo-images.com/bjdWaEjC0XleaCDWF...
134737,67629,U-Build Connect 4,2010,2,2,0,0,6,33,4.18182,...,,Children's Games,,1||Best~0|Recommended~0|Not Recommended~3|||2|...,,ccmonter|JUNK BIN\n\nMost of the entertainment...,,,https://cf.geekdo-images.com/agEIfeC82HLerQVQ9...,https://cf.geekdo-images.com/agEIfeC82HLerQVQ9...
134738,304430,Boney the Dinosaur,2015,4,8,0,0,0,0,0,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,https://cf.geekdo-images.com/LVPg9GMlSFzt5csLF...,https://cf.geekdo-images.com/LVPg9GMlSFzt5csLF...


## Function for queries

In [490]:
def get_df_from_db(query: str, out=False) -> pd.DataFrame:
    if out:
        conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

    df = pd.read_sql(query, conn)

    if out:
        conn.close()
    return df

def get_data_from_db(query: str, value=None, out=False, conn=False):
    if out:
        conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')
    cursor = conn.cursor()
    
    if value:
        cursor.execute(query, value)
    else:
        cursor.execute(query)

    data = cursor.fetchall()

    if out:
        conn.close()
    return data

## Prepare users data

In [491]:
def get_user_state(x):
    try:
        return x.split('|')[1]
    except:
        return np.NaN


def get_user_city(x):
    try:
        return x.split('|')[2]
    except:
        return np.NaN

def isNaN(num):
    return num != num

users['state_cuty'] = users.state_cuty.apply(lambda x: x.replace('<br/>', '|').replace('</div>', ''))
users['city'] = users.state_cuty.apply(get_user_state)
users['state'] = users.state_cuty.apply(get_user_city)
users['link'] = users.link.apply(lambda x: 'https://boardgamegeek.com' + x)
users

,link,nickname,country,state_cuty,city,state
0,https://boardgamegeek.com/user/ me,me,England,England,NaN,NaN
1,https://boardgamegeek.com/user/00296,00296,England,England|US Armed Forces - Europe,US Armed Forces - Europe,NaN
2,https://boardgamegeek.com/user/08bhattrell,08bhattrell,England,England,NaN,NaN
3,https://boardgamegeek.com/user/08pm,08pm,England,England|Michigan,Michigan,NaN
4,https://boardgamegeek.com/user/100pcBlade,100pcBlade,England,England|Crawley|West Sussex,Crawley,West Sussex
...,...,...,...,...,...,...
1665199,https://boardgamegeek.com/user/UndergroundMan,UndergroundMan,Tanzania,Tanzania|Unknown Village|Arusha Region,Unknown Village,Arusha Region
1665200,https://boardgamegeek.com/user/vortexturd,vortexturd,Tanzania,Tanzania,NaN,NaN
1665201,https://boardgamegeek.com/user/woodwardmw,woodwardmw,Tanzania,Tanzania|Dar es Salaam|Dar es Salaam,Dar es Salaam,Dar es Salaam
1665202,https://boardgamegeek.com/user/X3N0,X3N0,Tanzania,Tanzania|Unknown Village|Arusha Region,Unknown Village,Arusha Region


In [492]:
len(users.nickname) - len(users.nickname.drop_duplicates())

2

In [493]:
users = users.drop(index=208405)
users = users.drop(index=199352)

In [494]:
users[users.nickname == 'Sau_']

,link,nickname,country,state_cuty,city,state
1543499,https://boardgamegeek.com/user/Sau_,Sau_,Sweden,Sweden|Malmö,Malmö,NaN


In [495]:
users.iloc[1543498]

link          https://boardgamegeek.com/user/Savsjo03
nickname                                     Savsjo03
country                                        Sweden
state_cuty            Sweden|Stockholm|Stockholms län
city                                        Stockholm
state                                  Stockholms län
Name: 1543500, dtype: object

In [774]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# execute the insert statements
for i in list(users.country.unique()):
    cursor.execute('INSERT INTO countries (country) VALUES (?)', (i,))
    
# for i in list(users.state.unique()):
#     if not isNaN(i):
#         cursor.execute('INSERT INTO states (state) VALUES (?)', (i,))

# for i in list(users.city.unique()):
#     if not isNaN(i):
#         cursor.execute('INSERT INTO cities (city) VALUES (?)', (i,))

        
for index, row in tqdm(users.iterrows()):
    if not isNaN(row.country):
        country = get_data_from_db(f"""
                SELECT country_id
                FROM countries 
                WHERE country = ?
                """, (row.country,), conn=conn)[0][0]
    else:
        country = np.nan

#     if not isNaN(row.state):
#         state = get_data_from_db(f"""
#                 SELECT state_id
#                 FROM states 
#                 WHERE state = ?
#                 """, (row.state,), conn=conn)[0][0]
#     else:
#         state = np.nan
        
#     if not isNaN(row.city):
#         city = get_data_from_db(f"""
#                 SELECT city_id
#                 FROM cities 
#                 WHERE city = ?
#                 """, (row.city,), conn=conn)[0][0]
#     else:
#         city = np.nan
        
    last_check = '2023-02-01'
    cursor.execute('INSERT INTO users (nickname, country_id, last_check) VALUES (?, ?, ?)', (row.nickname, country, last_check)) #, state_id, city_id #, state, city
#     break
# commit the changes
conn.commit()

# close the connection
conn.close()

## Prepare boardgame data

In [497]:
def split_once(data: list) -> list:
    splitted = []
    for i in data:
        splitted += i.split('|')
    return list(set(splitted))


In [775]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# execute the insert statements
for i in list(set(list(boardgames.thumbnail_link.unique()) + list(boardgames.image_link.unique()))):
    if not isNaN(i):
        if i != '':
            cursor.execute('INSERT INTO links (link) VALUES (?)', (i,))

for i in split_once(list(set(boardgames.publishers[boardgames.publishers != '']))):
    if not isNaN(i):
            cursor.execute('INSERT INTO publishers (publisher) VALUES (?)', (i,)) 

# commit the changes
conn.commit()

# close the connection
conn.close()

In [776]:
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')
cursor = conn.cursor()
i = 'Joachim Zischke | terra authentica'

# cursor.execute('INSERT INTO publishers (publisher) VALUES (?)', (i,))
cursor.execute('''
SELECT publisher_id
FROM publishers
WHERE publisher = ?
LIMIT 10
''', (i,))


data = cursor.fetchall()
# conn.commit()
conn.close()
data

[]

In [777]:
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')
cursor = conn.cursor()
i = 'Joachim Zischke '

# cursor.execute('INSERT INTO publishers (publisher) VALUES (?)', (i,))
cursor.execute('''
DELETE FROM publishers
WHERE publisher = ?;
''', (i,))
i = ' terra authentica'
cursor.execute('''
DELETE FROM publishers
WHERE publisher = ?;
''', (i,))
i = 'Joachim Zischke | terra authentica'
cursor.execute('INSERT INTO publishers (publisher) VALUES (?)', (i,))
# data = cursor.fetchall()
conn.commit()
conn.close()
# data

In [778]:
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')
cursor = conn.cursor()
i = 'Joachim Zischke | terra authentica'

# cursor.execute('INSERT INTO publishers (publisher) VALUES (?)', (i,))
cursor.execute('''
SELECT publisher_id
FROM publishers
WHERE publisher = ?
LIMIT 10
''', (i,))


data = cursor.fetchall()
# conn.commit()
conn.close()
data

[(24080,)]

## Main pool of boardgames

In [779]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# execute the insert statements

for index, row in tqdm(boardgames[boardgames.year_published != ""].iterrows()):
    try:
        if row.year_published != '0': # year_published
            year_published = int(row.year_published)
        else:
            year_published = np.nan

        if row.minplayers != '0': # min_players
            min_players = int(row.minplayers)
        else:
            min_players = np.nan

        if int(row.maxplayers) < min_players: # max_players
            max_players = np.nan
        elif row.maxplayers == '0':
            max_players = min_players
        else:
            max_players = int(row.maxplayers)

        if row.minplaytime != '0': # min_playtime
            min_playtime = int(row.minplaytime)
        else:
            min_playtime = np.nan

        if int(row.maxplaytime) < min_playtime: # max_playtime
            max_playtime = np.nan
        elif row.maxplaytime == '0':
            max_playtime = min_playtime
        else:
            max_playtime = int(row.maxplaytime)

        if (row.age != '0') & (row.age != ''): # age
            age = int(row.age)
        else:
            age = np.nan

        if row.users_rated != '': # users_rated
            users_rated = int(row.users_rated)
        else:
            users_rated = np.nan

        if row.average_rating != '': # average_rating
            average_rating = float(row.average_rating)
        else:
            average_rating = np.nan

        if row.bayes_average_rating != '': # bayes_average_rating
            bayes_average_rating = float(row.bayes_average_rating)
        else:
            bayes_average_rating = np.nan

        if row.stddev != '': # stddev
            std_dev = float(row.stddev)
        else:
            std_dev = np.nan

        if row.owned != '': # owned
            owned = int(row.owned)
        else:
            owned = np.nan 

        if row.trading != '': # trading
            trading = int(row.trading)
        else:
            trading = np.nan

        if row.wishing != '': # wishing
            wishing = int(row.wishing)
        else:
            wishing = np.nan 

        if row.num_of_comments != '': # num_of_comments
            num_of_comments = int(row.num_of_comments)
        else:
            num_of_comments = np.nan

        if row.num_of_weights != '': # num_of_weights
            num_of_weights = int(row.num_of_weights)
        else:
            num_of_weights = np.nan

        if row.average_weight != '': # average_weight
            average_weight = float(row.average_weight)
        else:
            average_weight = np.nan

        if (isNaN(row.main_publisher)) | (row.main_publisher == ''): # main_publisher_id
            main_publisher_id = np.nan
        else:
            cursor.execute('''
                            SELECT *
                            FROM publishers
                            WHERE publisher = ?
                            ''',(row.main_publisher, ))
            main_publisher_id = cursor.fetchall()[0][0]

    #         main_publisher_id = get_data_from_db(f"""
    #                 SELECT publisher_id
    #                 FROM publishers 
    #                 WHERE publisher = ?
    #                 """, (row.main_publisher,), conn=conn)[0][0]

        if (isNaN(row.thumbnail_link)) | (row.thumbnail_link == ''): # thumbnail_link_id
            thumbnail_link_id = np.nan

        else:
            cursor.execute('''
                            SELECT *
                            FROM links
                            WHERE link = ?
                            ''',(row.thumbnail_link, ))
            thumbnail_link_id = cursor.fetchall()[0][0]
    #         thumbnail_link_id = get_data_from_db(f"""
    #                 SELECT link_id
    #                 FROM links 
    #                 WHERE link = ?
    #                 """, (row.thumbnail_link,), conn=conn)[0][0]

        if (isNaN(row.image_link)) | (row.image_link == ''): # image_link_id
            image_link_id = np.nan

        else:
            cursor.execute('''
                            SELECT *
                            FROM links
                            WHERE link = ?
                            ''',(row.image_link, ))
            image_link_id = cursor.fetchall()[0][0]
    #         image_link_id = get_data_from_db(f"""
    #                 SELECT link_id
    #                 FROM links 
    #                 WHERE link = ?
    #                 """, (row.image_link,), conn=conn)[0][0]

    #     print((int(row.boardgame_id), row.title, year_published, min_players, max_players, 
    #            min_playtime, max_playtime, age, users_rated, average_rating, bayes_average_rating, std_dev,
    #            owned, trading, wishing, num_of_comments, 
    #            num_of_weights, average_weight, main_publisher_id, thumbnail_link_id, image_link_id))
        cursor.execute('''INSERT INTO boardgames (boardgame_id, title, year_published, min_players, max_players, 
           min_playtime, max_playtime, age, users_rated, average_rating, bayes_average_rating, std_dev,
           owned, trading, wishing, num_of_comments, num_of_weights, average_weight, main_publisher_id, 
           thumbnail_link_id, image_link_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                       (int(row.boardgame_id), row.title, year_published, min_players, max_players, 
               min_playtime, max_playtime, age, users_rated, average_rating, bayes_average_rating, std_dev,
               owned, trading, wishing, num_of_comments, 
               num_of_weights, average_weight, main_publisher_id, thumbnail_link_id, image_link_id))

    except:
        print(int(row.boardgame_id))
# commit the changes
conn.commit()

# close the connection
conn.close()

0it [00:00, ?it/s]

In [503]:
boardgames[boardgames.boardgame_id.isin(['10983', '12767', '12768'])]#.main_publisher.iloc[0]

,boardgame_id,title,year_published,minplayers,maxplayers,minplaytime,maxplaytime,age,users_rated,average_rating,...,graphic_designers,subdomains,implementations,suggested_numplayers,podcast_episodes,comments,marketplace_history,marketplace_listings,thumbnail_link,image_link
43603,12768,Caraci: Das Testament,2004,2,4,30,30,0,0,0,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,
70158,10983,Landwehr 1438,2004,2,4,45,45,11,5,5.1,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,"JasonMatthews|Beautiful, but there's some old ...",unix_saledate|1478473003||saletime|94576682||p...,,https://cf.geekdo-images.com/VLRAFPYrDkcUgMWIA...,https://cf.geekdo-images.com/VLRAFPYrDkcUgMWIA...
94543,12767,Basatiin al Djabriin,2004,2,4,75,75,0,0,0,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,https://cf.geekdo-images.com/zWCrW9UroKZBSKXzV...,https://cf.geekdo-images.com/zWCrW9UroKZBSKXzV...


In [504]:
ratings[ratings.boardgame_id == 10983] # 12768 10983 12767

,nickname,title,boardgame_id,rating,num_of_plays,comment,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,last_modified
3197296,androgeus,Landwehr 1438,10983,6.0,1,None,0,1,0,0,0,0,0,0,2019-09-02 07:43:05
4646333,Banker,Landwehr 1438,10983,4.0,0,None,0,0,0,0,0,0,0,0,2010-01-09 11:28:12
7170911,JasonMatthews,Landwehr 1438,10983,2.0,1,"Beautiful, but there's some old saying about b...",0,0,0,0,0,0,0,0,2004-07-11 00:03:01
9532371,Windmilling,Landwehr 1438,10983,7.0,0,None,1,0,0,0,0,0,0,0,2018-01-05 10:24:44


In [505]:
ratings.iloc[0].comment == None

True

In [506]:
isNaN(ratings.iloc[0].comment)

False

## Prepare ratings data

In [507]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# execute the insert statements
for i in tqdm(list(ratings.comment.unique())[1:]):
    if not isNaN(i):
#         print(i)
        cursor.execute('INSERT INTO comments (comment) VALUES (?)', (i,))

# commit the changes
conn.commit()

# close the connection
conn.close()   

  0%|          | 0/3363581 [00:00<?, ?it/s]

In [508]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

for index, row in tqdm(boardgames[(boardgames.description != '') & (boardgames.year_published != "")].iterrows()):
#     row.boardgame_id , row.description
    cursor.execute('''
                    SELECT boardgame_id
                    FROM boardgames
                    WHERE boardgame_id = ?
                    ''',(int(row.boardgame_id), ))
    if cursor.fetchall() == []:
        print(row.boardgame_id)
    else:
        cursor.execute('''
                    INSERT INTO descriptions (boardgame_id, description) VALUES (?, ?)
                    ''',(int(row.boardgame_id), row.description))

# commit the changes
conn.commit()

# close the connection
conn.close()


0it [00:00, ?it/s]

Add missed nicknames

In [573]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

cursor.execute('''
        SELECT nickname
        FROM users
        ''')
data = cursor.fetchall()
# close the connection
conn.close()

In [574]:
users_from_db = []
for i in data:
    users_from_db.append(i[0])

In [575]:
list(set(ratings.nickname) - set(users_from_db))

['FaolanLuna',
 'alaxmo13',
 'horstderadler',
 'Boardgameplayer1234',
 'EastCoast',
 'Glasgow17',
 'DarkSideKitty']

In [576]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

for nickname in list(set(ratings.nickname.unique()) - set(users_from_db)):
    cursor.execute('''
    INSERT INTO users (nickname, country_id, website_id, registration_date, last_profile_update, last_login) VALUES (?, ?, ?, ?, ?, ?)
    ''',(nickname, np.nan, np.nan, '2022-01-01', '2022-01-01', '2022-01-01'))

# commit the changes
conn.commit()

# close the connection
conn.close()

## Additional boardgames

Update missed boardgames

In [587]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

cursor.execute('''
        SELECT comment
        FROM comments
        ''')
data = cursor.fetchall()
# close the connection
conn.close()

comments_from_db = []
for i in data:
    comments_from_db.append(i[0])

In [591]:
set(ratings.comment) - set(comments_from_db)

{None}

In [592]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

cursor.execute('''
        SELECT boardgame_id
        FROM boardgames
        ''')
data = cursor.fetchall()
# close the connection
conn.close()

ids = []
for i in data:
    ids.append(i[0])

In [610]:
len((set(ratings.boardgame_id) | set(boardgames.boardgame_id.astype(int))) - set(ids))

73

In [64]:
def get_api_bgg_game_data(unique_ids):
    '''
    This function retrieves aggregated game information via the boardgamegeek.com API
    
    ----------
    
    unique_ids - list of unique boardgames ids (it is best to feed a list of no more than 50 id)
    
    ----------
    Using API BoardGameGeek:
    https://api.geekdo.com/xmlapi/boardgame/37111?stats=1&pricehistory=1&marketplace=1&comments=1

    base - https://api.geekdo.com/xmlapi/boardgame
    game - /37111 - gameid
    params - ?stats=1&pricehistory=1&marketplace=1&comments=1
    comments: Show brief user comments on games (set it to 1, absent by default)
    stats: Include game statistics (set it to 1, absent by default)
    historical: Include historical game statistics (set it to 1, absent by default) - Use from/end parameters to set starting and ending dates. Returns all data starting from 2006-03-18.
    from: Set the start date to include historical data (format: YYYY-MM-DD, absent by default )
    to: Set the end date to include historical data (format: YYYY-MM-DD, absent by default )
    pricehistory: retrieve the marketplace history for this item (set it to 1, absent by default)
    marketplace: retrieve the current marketplace listings (set it to 1, absent by default)
    '''
    url_id = ''
    for i in range(len(unique_ids)):
        if i == 0:
            url_id = str(unique_ids[i])
        else:
            url_id += ',' + str(unique_ids[i])

    api_boardgame = 'https://api.geekdo.com/xmlapi/boardgame/'
    api_params = '?stats=1&pricehistory=1&marketplace=1&comments=1'
    r = re.get(api_boardgame + url_id + api_params)
    soup = BeautifulSoup(r.text, 'xml')
    list_bg = soup.find_all('boardgame')
    print(len(list_bg), len(unique_ids))
#     for bg in list_bg:
#         boardgame_id = bg.get('objectid')
#         dir_path = '../Board-Game-Data/scrapped/bgg_boardgame'
# #         dir_path = './data/scraped/'
#         with open(dir_path + '/' + str(boardgame_id) + '.xml', 'w', encoding='utf-8') as f:
#             f.write(str(bg))
    return list_bg

In [617]:
scrapped_bgs = get_api_bgg_game_data(list((set(ratings.boardgame_id) | set(boardgames.boardgame_id.astype(int))) - set(ids)))


139 73


In [649]:
BGG_ids = []
BGG_null = []
BGG_bg = []
for i in scrapped_bgs:
    if len(i) > 1:
        BGG_ids.append(int(i.get('objectid')))
        BGG_bg.append(i)
for i in scrapped_bgs:
    if len(i) == 1:
        BGG_null.append(int(i.get('objectid')))

In [656]:
scrapped_bgs = get_api_bgg_game_data(list((set(ratings.boardgame_id) | set(boardgames.boardgame_id.astype(int))) - set(ids) - set(BGG_ids)))


1 31


Check additional games in server

In [708]:
def get_api_bgg_game_data(list_):
    '''
    This function retrieves aggregated game information via the boardgamegeek.com API
    
    ----------
    ----------
    Using API BoardGameGeek:
    https://api.geekdo.com/xmlapi/boardgame/37111?stats=1&pricehistory=1&marketplace=1&comments=1

    base - https://api.geekdo.com/xmlapi/boardgame
    game - /37111 - gameid
    params - ?stats=1&pricehistory=1&marketplace=1&comments=1
    comments: Show brief user comments on games (set it to 1, absent by default)
    stats: Include game statistics (set it to 1, absent by default)
    historical: Include historical game statistics (set it to 1, absent by default) - Use from/end parameters to set starting and ending dates. Returns all data starting from 2006-03-18.
    from: Set the start date to include historical data (format: YYYY-MM-DD, absent by default )
    to: Set the end date to include historical data (format: YYYY-MM-DD, absent by default )
    pricehistory: retrieve the marketplace history for this item (set it to 1, absent by default)
    marketplace: retrieve the current marketplace listings (set it to 1, absent by default)
    '''
#     kk=0
    games = pd.DataFrame()
#     path = '../Board-Game-Data/scraped/bgg_boardgame/'
#     files = os.listdir(path)
    for bg in tqdm(list_):
#         f = open(path + bg_file, 'r')
#         bg = BeautifulSoup(bg, 'xml')
        try:
#             bg = bg.find('boardgame')
        #     get basic info
            boardgame_id = bg.get('objectid')
            try:
                thumbnail_link = bg.find('thumbnail').text 
            except:
                thumbnail_link = ""
            try:
                image_link = bg.find('image').text 
            except:
                image_link = ""
            try:
                year_published = bg.find('yearpublished').text
            except:
                year_published = ""
            try:
                minplayers = bg.find('minplayers').text
            except:
                minplayers = ""
            try:
                maxplayers = bg.find('maxplayers').text
            except:
                maxplayers = ""
            try:
                minplaytime = bg.find('minplaytime').text
            except:
                minplaytime = ""
            try:
                maxplaytime = bg.find('maxplaytime').text
            except:
                maxplaytime = ""
            try:
                title = bg.find('name', primary='true').text
            except:
                title = ""
            try:
                age = bg.find('age').text
            except:
                age = ""
            try:
                description = bg.find('description').text
            except:
                description = ""
            try:
                main_publisher = bg.find('boardgamepublisher').text
            except:
                main_publisher = ""


            publishers = []
            try:
                for i in bg.find_all('boardgamepublisher'):
                    publishers.append(i.text)
                publishers = '|'.join(publishers)
            except:
                publishers = ''
            honors = []
            try:
                for i in bg.find_all('boardgamehonor'):
                    honors.append(i.text)
                honors = '|'.join(honors)
            except:
                honors = ''
            expansions = []
            try:
                for i in bg.find_all('boardgameexpansion'):
                    expansions.append(i.text)
                expansions = '|'.join(expansions)
            except:
                expansions = ''
            accessories = []
            try:
                for i in bg.find_all('boardgameaccessory'):
                    accessories.append(i.text)
                accessories = '|'.join(accessories)
            except:
                accessories = ''
            artists = []
            try:
                for i in bg.find_all('boardgameartist'):
                    artists.append(i.text)
                artists = '|'.join(artists)
            except:
                artists = ''
            mechanics = []
            try:
                for i in bg.find_all('boardgamemechanic'):
                    mechanics.append(i.text)
                mechanics = '|'.join(mechanics)
            except:
                mechanics = ''
            category = []
            try:
                for i in bg.find_all('boardgamecategory'):
                    category.append(i.text)
                category = '|'.join(category)
            except:
                category = ''
            podcast_episodes = []
            try:
                for i in bg.find_all('boardgamepodcastepisode'):
                    podcast_episodes.append(i.text)
                podcast_episodes = '|'.join(podcast_episodes)
            except:
                podcast_episodes = ''
            designers = []
            try:
                for i in bg.find_all('boardgamedesigner'):
                    designers.append(i.text)
                designers = '|'.join(designers)
            except:
                designers = ''
            graphic_designers = []
            try:
                for i in bg.find_all('boardgamegraphicdesigner'):
                    graphic_designers.append(i.text)
                graphic_designers = '|'.join(graphic_designers)
            except:
                graphic_designers = ''
            subdomains = []
            try:
                for i in bg.find_all('boardgamesubdomain'):
                    subdomains.append(i.text)
                subdomains = '|'.join(subdomains)
            except:
                subdomains = ''
            implementations = []
            try:
                for i in bg.find_all('boardgameimplementation'):
                    implementations.append(i.text)
                implementations = '|'.join(implementations)
            except:
                implementations = ''

        #     get info about voting
            suggested_numplayers = []
            try:
                for i in bg.find('poll').find_all('results'):
                    numplayers = i.get('numplayers')
                    dict_vote = []
                    for f in i.find_all('result'):
        #                     dict_vote[f.get('value')] = f.get('numvotes')
                        dict_vote.append('~'.join((str(f.get('value')),str(f.get('numvotes')))))
        #                 suggested_numplayers[numplayers] = dict_vote
                    dict_vote = "|".join(dict_vote)
                    suggested_numplayers.append("||".join((str(numplayers),str(dict_vote))))
                suggested_numplayers = "|||".join(suggested_numplayers)
            except:
                suggested_numplayers = ''

        #     get all shorts comments
            comments = []
            try:
                for comment in bg.find_all('comment'):
        #                 comments[comment.get('username')] = comment.text
                    comments.append('|'.join((str(comment.get('username')), str(comment.text))))
                comments = '||'. join(comments)
            except:
                comments = ''

        #     get statistics info
            stats = bg.find('statistics')
            try:
                users_rated = stats.find('usersrated').text
            except:
                users_rated = ''
            try:
                average_rating = stats.find('average').text
            except:
                average_rating = ''
            try:
                bayes_average_rating = stats.find('bayesaverage').text
            except:
                bayes_average_rating = ''
            try:
                median = stats.find('median').text
            except:
                median = ''
            try:
                stddev = stats.find('stddev').text
            except:
                stddev = ''
            try:
                owned = stats.find('owned').text
            except:
                owned = ''
            try:
                trading = stats.find('trading').text
            except:
                trading = ''
            try:
                wishing = stats.find('wishing').text
            except:
                wishing = ''
            try:
                num_of_comments = stats.find('numcomments').text
            except:
                num_of_comments = ''
            try:
                num_of_weights = stats.find('numweights').text
            except:
                num_of_weights = ''
            try:
                average_weight = stats.find('averageweight').text
            except:
                average_weight = ''
            ranks = []
            try:
                ranking = stats.find_all('rank')
                for rank in ranking:
        #                 ranks[rank.get('friendlyname')] = rank.get('value')
                    ranks.append('|'.join((str(rank.get('friendlyname')), str(rank.get('value')))))
                ranks = '||'.join(ranks)
            except:
                ranks = ''

        #     get historical sales data
            try:
                sales_history = bg.find('marketplacehistory')
                listings = sales_history.find_all('listing')
                date_format = '%a, %d %b %Y %H:%M:%S +0000'
                marketplace_history = []
                for i, listing in enumerate(listings):
                    listdate = datetime.strptime(listing.find('listdate').text, date_format)
                    saledate = datetime.strptime(listing.find('saledate').text, date_format)
                    delta = saledate - listdate
                    saletime = delta.days*24*60*60 + delta.seconds
                    unix_saledate = int(datetime.timestamp(saledate))

                    price = listing.find('price').text
                    currency = listing.find('price').get('currency')
                    condition = listing.find('condition').text

        #                 marketplace_history[i] = {'unix_saledate':unix_saledate, 'saletime':saletime, 
        #                                          'price':price, 'currency':currency, 'condition':condition}
                    marketplace_history.append(f'unix_saledate|{unix_saledate}||saletime|{saletime}||price|{price}||currency|{currency}||condition|{condition}')
                marketplace_history = '|||'.join(marketplace_history)
            except:
                marketplace_history = ''


        #     get real sales data
            try:
                sales_history = bg.find('marketplacelistings')
                listings = sales_history.find_all('listing')
                marketplace_listings = {}
                for i, listing in enumerate(listings):
                    listdate = datetime.strptime(listing.find('listdate').text, date_format)
                    unix_listdate = int(datetime.timestamp(listdate))

                    price = listing.find('price').text
                    currency = listing.find('price').get('currency')
                    condition = listing.find('condition').text

        #                 marketplace_listings[i] = {'unix_listdate':unix_listdate,
        #                                          'price':price, 'currency':currency, 'condition':condition}
                    marketplace_listings[i] = f'unix_listdate|{unix_listdate}||price|{price}||currency|{currency}||condition|{condition}'
                    marketplace_listings = '|||'.join(marketplace_listings)
            except:
                marketplace_listings = ''

            game = {'boardgame_id':boardgame_id, 'title': title, 'year_published':year_published, 'minplayers':minplayers, 'maxplayers':maxplayers, 
                    'minplaytime': minplaytime, 'maxplaytime':maxplaytime, 'age':age, 'users_rated':users_rated, 
                    'average_rating':average_rating, 'bayes_average_rating':bayes_average_rating, 'median':median, 
                    'stddev':stddev, 'owned':owned, 'trading':trading, 'wishing':wishing, 'num_of_comments':num_of_comments, 
                    'num_of_weights':num_of_weights, 'average_weight':average_weight, 'ranks':ranks, 
                    'main_publisher':main_publisher, 
                    'description':description, 'publishers':publishers, 'honors':honors, 'expansions':expansions, 
                    'accessories':accessories, 'artists':artists, 'mechanics':mechanics, 'category':category, 
                    'designers':designers, 'graphic_designers':graphic_designers, 'subdomains':subdomains, 
                    'implementations':implementations, 'suggested_numplayers':suggested_numplayers,
                    'podcast_episodes':podcast_episodes, 'comments':comments, 'marketplace_history':marketplace_history, 
                    'marketplace_listings':marketplace_listings, 'thumbnail_link': thumbnail_link, 'image_link': image_link
                   }
            games = pd.concat([games, pd.DataFrame([game])])
        except:
            pass
#         kk += 1
#         if kk>2: 
#             break
    return games.reset_index(drop=True)
x = get_api_bgg_game_data(BGG_bg)
x

  0%|          | 0/42 [00:00<?, ?it/s]

,boardgame_id,title,year_published,minplayers,maxplayers,minplaytime,maxplaytime,age,users_rated,average_rating,...,graphic_designers,subdomains,implementations,suggested_numplayers,podcast_episodes,comments,marketplace_history,marketplace_listings,thumbnail_link,image_link
0,25729,World at War: Eisenbach Gap,2007,2,2,90,90,12,660,7.4713,...,,Wargames,World At War 85: Storming the Gap,1||Best~3|Recommended~16|Not Recommended~3|||2...,"GrogCast Season 2, Episode 8|Mentioned in Disp...",100pcBlade|Not played Yet||abeheron|Oh how I w...,,,https://cf.geekdo-images.com/jkOTmDwdgKcsnM1hM...,https://cf.geekdo-images.com/jkOTmDwdgKcsnM1hM...
1,158466,Captain's Log #37,,,,,,,0,0,...,,,,+||Best~0|Recommended~0|Not Recommended~0,,,,,https://cf.geekdo-images.com/YDYs_tg3G7e18u2Jo...,https://cf.geekdo-images.com/YDYs_tg3G7e18u2Jo...
2,141060,Captain's Log #18,,,,,,,1,10,...,,,,+||Best~0|Recommended~0|Not Recommended~0,,,,,https://cf.geekdo-images.com/1klrsQDCmYgwJGAGc...,https://cf.geekdo-images.com/1klrsQDCmYgwJGAGc...
3,35462,Lock 'n Load: Noville – Bastogne's Outpost,2009,2,2,60,60,14,104,8.10844,...,,Wargames,,1||Best~0|Recommended~3|Not Recommended~0|||2|...,,ariel_d|OC300-09\nAcquired 2017 from Robert Fi...,,,https://cf.geekdo-images.com/VGMdWwSXmGpHyGNvt...,https://cf.geekdo-images.com/VGMdWwSXmGpHyGNvt...
4,158468,Dispatches From the Bunker #37,,,,,,,0,0,...,,,,+||Best~0|Recommended~0|Not Recommended~0,,,,,,
5,6542,Lock 'n Load: Forgotten Heroes – Vietnam,2003,2,2,60,60,12,361,7.40083,...,,Wargames,Lock 'n Load Tactical: Heroes of the Nam,1||Best~0|Recommended~7|Not Recommended~0|||2|...,The Gaming Gang Episode 34: Consensus - Movie ...,-E..|Unpunched - 1st Edition||832SPS|1965||abr...,,,https://cf.geekdo-images.com/AYvxpJFgIgXEBnC1V...,https://cf.geekdo-images.com/AYvxpJFgIgXEBnC1V...
6,18460,Lock 'n Load: Band of Heroes,2005,2,2,60,60,12,629,7.39011,...,,Wargames,Lock 'n Load Tactical: Heroes of Normandy,1||Best~3|Recommended~13|Not Recommended~1|||2...,"GrogCast Season 1, The Lost Episode",-E..|Unpunched - 1st Edition||100pcBlade|Sold ...,,,https://cf.geekdo-images.com/1FkQTCz204asibe7s...,https://cf.geekdo-images.com/1FkQTCz204asibe7s...
7,141213,Dispatches From the Bunker #36,,,,,,,0,0,...,,,,+||Best~0|Recommended~0|Not Recommended~0,,Mark_WH|ASL,,,,
8,36,Federation & Empire,1986,2,8,300,300,12,417,6.40743,...,,Wargames,Federation Space,1||Best~0|Recommended~2|Not Recommended~3|||2|...,"I've Been Diced! episode 45: Noble failures, a...",-E..|2010 Edition - Unpunched||11B4V|Will alwa...,,,https://cf.geekdo-images.com/ZlZQnkIDefuLQMJGX...,https://cf.geekdo-images.com/ZlZQnkIDefuLQMJGX...
9,141098,Captain's Log #38,,,,,,,0,0,...,,,,+||Best~0|Recommended~0|Not Recommended~0,,,,,https://cf.geekdo-images.com/HBU8pwdYdj-cYiMMW...,https://cf.geekdo-images.com/HBU8pwdYdj-cYiMMW...


In [711]:
# x.to_feather('../add_bg.feather')

## Links

Add links in DB

In [746]:
len(list(set(list(x.thumbnail_link) + list(x.image_link))))

75

In [780]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()
flag = []
# execute the insert statements
for i in list(set(list(x.thumbnail_link) + list(x.image_link))):
    if i != '':
#         cursor.execute('SELECT * FROM links WHERE link = ?', (i,))
#         if cursor.fetchall() == []:
        try:
            cursor.execute('INSERT INTO links (link) VALUES (?)', (i,))
            print('Add link -> ', i)
            
        except:
            flag.append(i)
# for i in split_once(list(set(boardgames.publishers[boardgames.publishers != '']))):
#     if not isNaN(i):
#             cursor.execute('INSERT INTO publishers (publisher) VALUES (?)', (i,)) 

# commit the changes
conn.commit()

# close the connection
conn.close()
len(flag)

Add link ->  https://cf.geekdo-images.com/q88VeafRDQbOmAvPxIIvrg__original/img/3tKL8qVm2aF4_K10BGg-yxDWFOA=/0x0/filters:format(jpeg)/pic3227431.jpg
Add link ->  https://cf.geekdo-images.com/1phjQQvE37PafqCmhVocmA__original/img/TAZ1yIVQiq8IKg2izN-9mR7kDys=/0x0/filters:format(jpeg)/pic2530503.jpg
Add link ->  https://cf.geekdo-images.com/1FkQTCz204asibe7s4E9mw__original/img/Oqo5s6QSsBGU0GagFvE1_QcB3GA=/0x0/filters:format(jpeg)/pic629266.jpg
Add link ->  https://cf.geekdo-images.com/Whte0ogVFty0bcqBMxL0wQ__original/img/QPJihMUOcjMzk8uQ0cP4GCHm0Aw=/0x0/filters:format(jpeg)/pic126450.jpg
Add link ->  https://cf.geekdo-images.com/peCd96QGOWVosggUeIz59w__original/img/tK9Q1lXFO6fYdqWPQ5r5FIzv5jE=/0x0/filters:format(jpeg)/pic499375.jpg
Add link ->  https://cf.geekdo-images.com/VGMdWwSXmGpHyGNvtlLonA__thumb/img/efxy4_UhPBsKPVf5VTshdsKGPL0=/fit-in/200x150/filters:strip_icc()/pic5350257.jpg
Add link ->  https://cf.geekdo-images.com/VGMdWwSXmGpHyGNvtlLonA__original/img/2YjdHvpqkeP051Bimie2bRbYKx8=/

40

In [764]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
for i in list(set(list(x.thumbnail_link) + list(x.image_link))):
    if i != '':
        cursor.execute('SELECT * FROM links WHERE link = ?', (i,))
#         print(cursor.fetchall()[0][0])

# close the connection
conn.close()

In [781]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# execute the insert statements

for index, row in tqdm(x.iterrows()):
#     try:
    try:
        if row.year_published != '0': # year_published
            year_published = int(row.year_published)
        else:
            year_published = np.nan
    except:
        year_published = np.nan
    
    try:
        if row.minplayers != '0': # min_players
            min_players = int(row.minplayers)
        else:
            min_players = np.nan
    except:
        min_players = np.nan
    
    try:
        if int(row.maxplayers) < min_players: # max_players
            max_players = np.nan
        elif row.maxplayers == '0':
            max_players = min_players
        else:
            max_players = int(row.maxplayers)
    except:
        max_players = np.nan
    
    try:
        if row.minplaytime != '0': # min_playtime
            min_playtime = int(row.minplaytime)
        else:
            min_playtime = np.nan
    except:
        min_playtime = np.nan
    
    try:
        if int(row.maxplaytime) < min_playtime: # max_playtime
            max_playtime = np.nan
        elif row.maxplaytime == '0':
            max_playtime = min_playtime
        else:
            max_playtime = int(row.maxplaytime)
    except:
        max_playtime = np.nan
    
    try:
        if (row.age != '0') & (row.age != ''): # age
            age = int(row.age)
        else:
            age = np.nan
    except:
        age = np.nan
    

    if row.users_rated != '': # users_rated
        users_rated = int(row.users_rated)
    else:
        users_rated = np.nan

    if row.average_rating != '': # average_rating
        average_rating = float(row.average_rating)
    else:
        average_rating = np.nan

    if row.bayes_average_rating != '': # bayes_average_rating
        bayes_average_rating = float(row.bayes_average_rating)
    else:
        bayes_average_rating = np.nan

    if row.stddev != '': # stddev
        std_dev = float(row.stddev)
    else:
        std_dev = np.nan

    if row.owned != '': # owned
        owned = int(row.owned)
    else:
        owned = np.nan 

    if row.trading != '': # trading
        trading = int(row.trading)
    else:
        trading = np.nan

    if row.wishing != '': # wishing
        wishing = int(row.wishing)
    else:
        wishing = np.nan 

    if row.num_of_comments != '': # num_of_comments
        num_of_comments = int(row.num_of_comments)
    else:
        num_of_comments = np.nan

    if row.num_of_weights != '': # num_of_weights
        num_of_weights = int(row.num_of_weights)
    else:
        num_of_weights = np.nan

    if row.average_weight != '': # average_weight
        average_weight = float(row.average_weight)
    else:
        average_weight = np.nan
            
    if (row.main_publisher == ''): # main_publisher_id
        main_publisher_id = np.nan
    else:
        cursor.execute('''
                        SELECT *
                        FROM publishers
                        WHERE publisher = ?
                        ''',(row.main_publisher, ))

        main_publisher_id = cursor.fetchall()[0][0]

#         main_publisher_id = get_data_from_db(f"""
#                 SELECT publisher_id
#                 FROM publishers 
#                 WHERE publisher = ?
#                 """, (row.main_publisher,), conn=conn)[0][0]

    if (row.thumbnail_link == ''): # thumbnail_link_id
        thumbnail_link_id = np.nan

    else:
        cursor.execute('''
                        SELECT *
                        FROM links
                        WHERE link = ?
                        ''',(row.thumbnail_link, ))
        thumbnail_link_id = cursor.fetchall()[0][0]
#         thumbnail_link_id = get_data_from_db(f"""
#                 SELECT link_id
#                 FROM links 
#                 WHERE link = ?
#                 """, (row.thumbnail_link,), conn=conn)[0][0]

    if (row.image_link == ''): # image_link_id
        image_link_id = np.nan

    else:
        cursor.execute('''
                        SELECT *
                        FROM links
                        WHERE link = ?
                        ''',(row.image_link, ))
        data = cursor.fetchall()
#         print(data[0][0])
        image_link_id = data[0][0]
#         image_link_id = get_data_from_db(f"""
#                 SELECT link_id
#                 FROM links 
#                 WHERE link = ?
#                 """, (row.image_link,), conn=conn)[0][0]

#     print((int(row.boardgame_id), row.title, year_published, min_players, max_players, 
#            min_playtime, max_playtime, age, users_rated, average_rating, bayes_average_rating, std_dev,
#            owned, trading, wishing, num_of_comments, 
#            num_of_weights, average_weight, main_publisher_id, thumbnail_link_id, image_link_id))
    cursor.execute('''INSERT INTO boardgames (boardgame_id, title, year_published, min_players, max_players, 
       min_playtime, max_playtime, age, users_rated, average_rating, bayes_average_rating, std_dev,
       owned, trading, wishing, num_of_comments, num_of_weights, average_weight, main_publisher_id, 
       thumbnail_link_id, image_link_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                   (int(row.boardgame_id), row.title, year_published, min_players, max_players, 
           min_playtime, max_playtime, age, users_rated, average_rating, bayes_average_rating, std_dev,
           owned, trading, wishing, num_of_comments, 
           num_of_weights, average_weight, main_publisher_id, thumbnail_link_id, image_link_id))
#     print(int(row.boardgame_id),'##', main_publisher_id,thumbnail_link_id, image_link_id,row.image_link ,'##')

#     except:
#         print(int(row.boardgame_id))
# commit the changes
conn.commit()

# close the connection
conn.close()

0it [00:00, ?it/s]

## Ratings

In [55]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
cursor.execute('SELECT boardgame_id FROM boardgames')
data = cursor.fetchall()


# close the connection
conn.close()

In [56]:
len(data)

134740

In [61]:
ids = []
for i in data:
    ids.append(i[0])

In [68]:
non_bg_ids = list(set(ratings.boardgame_id) - set(ids))

In [75]:
ratings[~ratings.boardgame_id.isin(non_bg_ids)].head()

,nickname,title,boardgame_id,rating,num_of_plays,comment,own,prevowned,fortrade,want,wanttoplay,wanttobuy,wishlist,preordered,last_modified
0,DiVo1975,Anachrony,185343,9.0,2,None,0,1,0,0,0,0,0,0,2022-11-06 14:02:30
1,DiVo1975,Anachrony: Infinity Box,278292,10.0,2,None,1,0,0,0,0,0,0,0,2022-01-06 06:21:59
2,DiVo1975,Ark Nova,342942,10.0,25,None,1,0,0,0,0,0,0,0,2022-09-05 01:21:16
3,DiVo1975,Arkham Horror: The Card Game,205637,9.0,0,None,1,0,0,0,0,0,0,0,2022-11-06 14:02:55
4,DiVo1975,Blood Rage,170216,10.0,1,None,1,0,0,0,0,0,0,0,2022-11-06 14:03:59


In [79]:
len(ratings[~ratings.boardgame_id.isin(non_bg_ids)])

20186551

In [102]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# # execute the insert statements
# for i in list(ratings.comment.unique())[1:]:
#     if not isNaN(i):
#         cursor.execute('INSERT INTO comments (comment) VALUES (?)', (i,))

for index, row in tqdm(ratings[~ratings.boardgame_id.isin(non_bg_ids)][~ratings.rating.isna()].iterrows()):
    cursor.execute('''
                SELECT user_id
                FROM users 
                WHERE nickname = ?
                ''',(row.nickname, ))
    user_id = cursor.fetchall()[0][0]
    
    boardgame_id = int(row.boardgame_id)
    rating = float(row.rating)
    num_of_plays = int(row.num_of_plays)
    
    if row.comment == None:
        comment_id = np.nan
    else:
        cursor.execute('''
                    SELECT comment_id
                    FROM comments 
                    WHERE comment = ?
                    ''',(row.comment, ))
        comment_id = cursor.fetchall()[0][0]
    
    own = int(row.own)
    prevowned = int(row.prevowned)
    for_trade = int(row.fortrade)
    want = int(row.want)
    want_to_play = int(row.wanttoplay)
    want_to_buy = int(row.wanttobuy)
    wishlist = int(row.wishlist)
    preordered = int(row.preordered)
    last_modified = row.last_modified[:10]
#     print(user_id, boardgame_id, rating, num_of_plays, comment_id, own, prevowned, for_trade, want,
#           want_to_play, want_to_buy, wishlist, preordered, last_modified)
    cursor.execute('''
                INSERT INTO ratings (user_id, boardgame_id, rating, num_of_plays, comment_id, own, prevowned, for_trade, want, want_to_play, want_to_buy, wishlist, preordered, last_modified) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''',(user_id, boardgame_id, rating, num_of_plays, comment_id, own, prevowned, for_trade, want,
                want_to_play, want_to_buy, wishlist, preordered, last_modified))
#     break


# # commit the changes
conn.commit()

# close the connection
conn.close()   

/var/folders/g8/brdkwrws1mj2h2g5gb9xv8p00000gn/T/ipykernel_56462/3837096943.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for index, row in tqdm(ratings[~ratings.boardgame_id.isin(non_bg_ids)][~ratings.rating.isna()].iterrows()):


0it [00:00, ?it/s]

## Additional Decriptions

Add gaps into boardgame descriptions table

In [6]:
 PATH_TO_DB = '../Board-Game-Data/board_games.sqlite'

In [8]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()

# Execute the PRAGMA statement to get the table schema
# cursor.execute("PRAGMA table_info(ratings)")

# Fetch all the rows returned by the statement
rows = cursor.fetchall()

# Print the table schema
for row in rows:
    print(row)

cursor.execute('''
        SELECT boardgame_id
        FROM boardgames
        EXCEPT
        SELECT boardgame_id
        FROM descriptions
        ''')
data = cursor.fetchall()
# close the connection
conn.close()

len(data)

42

In [16]:
ids = []

for i in data:
    ids.append(str(i[0]))

In [18]:
x = pd.read_feather('../add_bg.feather')

In [25]:
x[x.boardgame_id.isin(ids)][['boardgame_id', 'description']]

,boardgame_id,description
0,25729,It's the summer of 1985 and Soviet tanks strea...
1,158466,"Basics: 120 pages, 6 scenarios, 11 new ships.<..."
2,141060,"Data: 112 pages, 7 scenarios, 16+3 new SSD.<br..."
3,35462,Lock 'n Load: Noville – Bastogne's Outpost Am...
4,158468,A magazine dedicated to Advanced Squad Leader:...
5,6542,Forgotten Heroes is now out of print and has b...
6,18460,"This game is no longer available, the new, imp..."
7,141213,A magazine dedicated to Advanced Squad Leader....
8,36,This sprawling empire game is set in the Star...
9,141098,"Basics: 120 pages, 6 scenarios, 23 new SSD.<br..."


In [ ]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
for index, row in x[x.boardgame_id.isin(ids)].iterrows():
#     print(int(row.boardgame_id), row.description)
    cursor.execute('''
        INSERT INTO descriptions (boardgame_id, description) VALUES (?, ?)
        ''', (int(row.boardgame_id), row.description))

# commit the changes
conn.commit()

# close the connection
conn.close()

## Awards

In [391]:
x

,boardgame_id,title,year_published,minplayers,maxplayers,minplaytime,maxplaytime,age,users_rated,average_rating,...,graphic_designers,subdomains,implementations,suggested_numplayers,podcast_episodes,comments,marketplace_history,marketplace_listings,thumbnail_link,image_link
0,25729,World at War: Eisenbach Gap,2007,2,2,90,90,12,660,7.4713,...,,Wargames,World At War 85: Storming the Gap,1||Best~3|Recommended~16|Not Recommended~3|||2...,"GrogCast Season 2, Episode 8|Mentioned in Disp...",100pcBlade|Not played Yet||abeheron|Oh how I w...,,,https://cf.geekdo-images.com/jkOTmDwdgKcsnM1hM...,https://cf.geekdo-images.com/jkOTmDwdgKcsnM1hM...
1,158466,Captain's Log #37,,,,,,,0,0,...,,,,+||Best~0|Recommended~0|Not Recommended~0,,,,,https://cf.geekdo-images.com/YDYs_tg3G7e18u2Jo...,https://cf.geekdo-images.com/YDYs_tg3G7e18u2Jo...
2,141060,Captain's Log #18,,,,,,,1,10,...,,,,+||Best~0|Recommended~0|Not Recommended~0,,,,,https://cf.geekdo-images.com/1klrsQDCmYgwJGAGc...,https://cf.geekdo-images.com/1klrsQDCmYgwJGAGc...
3,35462,Lock 'n Load: Noville – Bastogne's Outpost,2009,2,2,60,60,14,104,8.10844,...,,Wargames,,1||Best~0|Recommended~3|Not Recommended~0|||2|...,,ariel_d|OC300-09\nAcquired 2017 from Robert Fi...,,,https://cf.geekdo-images.com/VGMdWwSXmGpHyGNvt...,https://cf.geekdo-images.com/VGMdWwSXmGpHyGNvt...
4,158468,Dispatches From the Bunker #37,,,,,,,0,0,...,,,,+||Best~0|Recommended~0|Not Recommended~0,,,,,,
5,6542,Lock 'n Load: Forgotten Heroes – Vietnam,2003,2,2,60,60,12,361,7.40083,...,,Wargames,Lock 'n Load Tactical: Heroes of the Nam,1||Best~0|Recommended~7|Not Recommended~0|||2|...,The Gaming Gang Episode 34: Consensus - Movie ...,-E..|Unpunched - 1st Edition||832SPS|1965||abr...,,,https://cf.geekdo-images.com/AYvxpJFgIgXEBnC1V...,https://cf.geekdo-images.com/AYvxpJFgIgXEBnC1V...
6,18460,Lock 'n Load: Band of Heroes,2005,2,2,60,60,12,629,7.39011,...,,Wargames,Lock 'n Load Tactical: Heroes of Normandy,1||Best~3|Recommended~13|Not Recommended~1|||2...,"GrogCast Season 1, The Lost Episode",-E..|Unpunched - 1st Edition||100pcBlade|Sold ...,,,https://cf.geekdo-images.com/1FkQTCz204asibe7s...,https://cf.geekdo-images.com/1FkQTCz204asibe7s...
7,141213,Dispatches From the Bunker #36,,,,,,,0,0,...,,,,+||Best~0|Recommended~0|Not Recommended~0,,Mark_WH|ASL,,,,
8,36,Federation & Empire,1986,2,8,300,300,12,417,6.40743,...,,Wargames,Federation Space,1||Best~0|Recommended~2|Not Recommended~3|||2|...,"I've Been Diced! episode 45: Noble failures, a...",-E..|2010 Edition - Unpunched||11B4V|Will alwa...,,,https://cf.geekdo-images.com/ZlZQnkIDefuLQMJGX...,https://cf.geekdo-images.com/ZlZQnkIDefuLQMJGX...
9,141098,Captain's Log #38,,,,,,,0,0,...,,,,+||Best~0|Recommended~0|Not Recommended~0,,,,,https://cf.geekdo-images.com/HBU8pwdYdj-cYiMMW...,https://cf.geekdo-images.com/HBU8pwdYdj-cYiMMW...


Fill data into awards table

In [ ]:
awards = list(x[x.honors != ''].honors.apply(lambda x: x.split('|')).explode().apply(lambda x: x[5:]).unique())
print(len(awards))
for i, award in enumerate(awards):
    try:
        int(award[:4])
        awards[i] = award[5:]
    except:
        pass
awards = list(set(awards))
len(awards)

In [63]:
awards = list(boardgames[boardgames.honors != ''].honors.apply(lambda x: x.split('|')).explode().apply(lambda x: x[5:]).unique())
print(len(awards))
for i, award in enumerate(awards):
    try:
        int(award[:4])
        awards[i] = award[5:]
    except:
        pass
awards = list(set(awards))
len(awards)

1152


1134

In [ ]:
awards = list(boardgames[boardgames.honors != ''].honors.apply(lambda x: x.split('|')).explode().apply(lambda x: x[5:]).unique())
print(len(awards))
for i, award in enumerate(awards):
    try:
        int(award[:4])
        awards[i] = award[5:]
    except:
        pass
awards = list(set(awards))
len(awards)

In [66]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
for i in awards:
#     print(int(row.boardgame_id), row.description)
    cursor.execute('''
        INSERT INTO awards (award) VALUES (?)
        ''', (i,))

# commit the changes
conn.commit()

# close the connection
conn.close()

## Honors

0     2007 Charles S. Roberts Best Post-WWII Era Boa...
1                                                      
2                                                      
3     2009 Charles S. Roberts Best Expansion or Supp...
4                                                      
5     2003 Charles S. Roberts Best Modern Era Boardg...
6     2005 Origins Awards Historical Board Game of t...
7                                                      
8                                                      
9                                                      
10                                                     
11                                                     
12                                                     
13    1982 Charles S. Roberts Best Science-Fiction B...
14                                                     
15                                                     
16                                                     
17                                              

In [96]:
honors = list(boardgames[boardgames.honors != ''].honors.apply(lambda x: x.split('|')).explode().unique())
print(len(honors))

4853


In [404]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)
xxhon = [(2007, 104), (2003, 529), (2004, 608), (2005, 541)]
# create a cursor object
cursor = conn.cursor()
# execute the insert statements
for i in xxhon:
    year = i[0]
    
    award_id = i[1]
    print(year, award_id)
    cursor.execute('''
        INSERT INTO honors (year, award_id) VALUES (?, ?)
        ''', (year, award_id))

# commit the changes
conn.commit()

# close the connection
conn.close()

2007 104
2003 529
2004 608
2005 541


In [ ]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
for i in honors:
    year = int(i[:4])
    if i[5:9].isdigit():
        award = i[10:]
    else: 
        award = i[5:]
    cursor.execute('''
        SELECT award_id
        FROM awards
        WHERE award = ?
        ''', (award,))
    award_id = cursor.fetchall()[0][0]
#     print(year, award_id, award)
    cursor.execute('''
        INSERT INTO honors (year, award_id) VALUES (?, ?)
        ''', (year, award_id))

# commit the changes
conn.commit()

# close the connection
conn.close()

## Boardgame_honors

In [ ]:
boardgames['split_honors'] = boardgames[boardgames.honors != ''].honors.apply(lambda x: x.split('|'))

In [431]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
# for i in honors:
for index, row in boardgames[boardgames.honors != ''][['boardgame_id', 'split_honors']].explode('split_honors').drop_duplicates().iterrows():
    i = row.split_honors
    year = int(i[:4])
    if i[5:9].isdigit():
        award = i[10:]
    else: 
        award = i[5:]
    cursor.execute('''
        SELECT award_id
        FROM awards
        WHERE award = ?
        ''', (award,))
    award_id = cursor.fetchall()[0][0]

    cursor.execute('''
        SELECT honor_id
        FROM honors
        WHERE year = ? AND award_id = ?
        ''', (year, award_id))
    honor_id = cursor.fetchall()[0][0]
    cursor.execute('''
        INSERT INTO boardgame_honors (boardgame_id, honor_id) VALUES (?, ?)
        ''', (int(row.boardgame_id), honor_id))

# commit the changes
conn.commit()

# close the connection
conn.close()

In [394]:
x['split_honors'] = x[x.honors != ''].honors.apply(lambda x: x.split('|'))

In [432]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
# for i in honors:
for index, row in x[x.honors != ''][['boardgame_id', 'split_honors']].explode('split_honors').drop_duplicates().iterrows():
    i = row.split_honors
    year = int(i[:4])
    if i[5:9].isdigit():
        award = i[10:]
    else: 
        award = i[5:]
    cursor.execute('''
        SELECT award_id
        FROM awards
        WHERE award = ?
        ''', (award,))
    try:
        award_id = cursor.fetchall()[0][0]
    except:
        print('Award :', award)
        award_id = np.NaN

    cursor.execute('''
        SELECT honor_id
        FROM honors
        WHERE year = ? AND award_id = ?
        ''', (year, award_id))
    try:
        honor_id = cursor.fetchall()[0][0]
    except:
        print('Honor :', year, award_id)
        honor_id = np.NaN
    cursor.execute('''
        INSERT INTO boardgame_honors (boardgame_id, honor_id) VALUES (?, ?)
        ''', (int(row.boardgame_id), honor_id))

# commit the changes
conn.commit()

# close the connection
conn.close()

## Boardgame_publishers

In [408]:
x.publishers

0                         Lock 'n Load Publishing, LLC.
1                          Amarillo Design Bureau, Inc.
2                          Amarillo Design Bureau, Inc.
3                         Lock 'n Load Publishing, LLC.
4                            Dispatches from the Bunker
5     Lock 'n Load Publishing, LLC.|Shrapnel Games, ...
6            Lock 'n Load Publishing, LLC.|Matrix Games
7                            Dispatches from the Bunker
8         Amarillo Design Bureau, Inc.|Task Force Games
9                          Amarillo Design Bureau, Inc.
10                         Amarillo Design Bureau, Inc.
11                         Amarillo Design Bureau, Inc.
12                         Amarillo Design Bureau, Inc.
13        Amarillo Design Bureau, Inc.|Task Force Games
14        Amarillo Design Bureau, Inc.|Task Force Games
15                                                     
16                         Amarillo Design Bureau, Inc.
17                         Amarillo Design Burea

In [149]:
boardgames['all_publishers'] = boardgames.publishers.str.split('|')
all_publishers = boardgames[['boardgame_id', 'all_publishers']].explode('all_publishers')

In [153]:
all_publishers[~((all_publishers.all_publishers == 'Joachim Zischke ') | (all_publishers.all_publishers == ' terra authentica'))]


,boardgame_id,all_publishers
0,5848,Arteeno Entertainment Company Ltd.
1,259048,Fantasy Flight Games
1,259048,Rebel Sp. z o.o.
2,122324,Spiel Spass
3,239964,Hasbro
...,...,...
134736,114857,Takara
134736,114857,THQ (dōjin circle)
134737,67629,Hasbro
134738,304430,Junkhouse Entertainment LLC


In [423]:
all_publishers[~((all_publishers.all_publishers == 'Joachim Zischke ') | (all_publishers.all_publishers == ' terra authentica'))].drop_duplicates()

,boardgame_id,all_publishers
0,5848,Arteeno Entertainment Company Ltd.
1,259048,Fantasy Flight Games
1,259048,Rebel Sp. z o.o.
2,122324,Spiel Spass
3,239964,Hasbro
...,...,...
134736,114857,Takara
134736,114857,THQ (dōjin circle)
134737,67629,Hasbro
134738,304430,Junkhouse Entertainment LLC


In [427]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# commit the changes
conn.commit()

# close the connection
conn.close()

In [434]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
# for i in honors:
for index, row in all_publishers[~((all_publishers.all_publishers == 'Joachim Zischke ') | (all_publishers.all_publishers == ' terra authentica'))].drop_duplicates().iterrows():
#     try:
    i = row.all_publishers
    if i != '':
        cursor.execute('''
            SELECT publisher_id
            FROM publishers
            WHERE publisher = ?
            ''', (i,))
        publisher_id = cursor.fetchall()[0][0]

        cursor.execute('''
            INSERT INTO boardgame_publishers (boardgame_id, publisher_id) VALUES (?, ?)
            ''', (int(row.boardgame_id), publisher_id))

# commit the changes
conn.commit()

# close the connection
conn.close()

In [435]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
# for i in honors:
for index, row in all_publishers[(all_publishers.all_publishers == 'Joachim Zischke ')].iterrows():
#     try:
    i = "Joachim Zischke | terra authentica"
    if i != '':
        cursor.execute('''
            SELECT publisher_id
            FROM publishers
            WHERE publisher = ?
            ''', (i,))
        publisher_id = cursor.fetchall()[0][0]

        cursor.execute('''
            INSERT INTO boardgame_publishers (boardgame_id, publisher_id) VALUES (?, ?)
            ''', (int(row.boardgame_id), publisher_id))
        print(int(row.boardgame_id), publisher_id)

# commit the changes
conn.commit()

# close the connection
conn.close()

12768 24080
10983 24080
12767 24080


## Accessories

In [250]:
accessories = list(set(boardgames.accessories[boardgames.accessories != ''].str.split('|').explode().to_list()))

In [444]:
list(set(x.accessories[x.accessories != ''].str.split('|').explode().to_list()) - set(accessories))

['Test item expansion to accessory',
 "Lock 'n Load: Noville – Bastogne's Outpost: X-Maps",
 'Admin Test Accessory',
 'Federation & Empire: Large Scale Map',
 'ASL GAP']

In [445]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
for i in accessories:
    if i != '':
#     print(int(row.boardgame_id), row.description)
        cursor.execute('''
            INSERT INTO accessories (accessory) VALUES (?)
            ''', (i,))

# commit the changes
conn.commit()

# close the connection
conn.close()

## Boardgame_accessories

In [260]:
boardgames['accessories_list'] = boardgames.accessories[boardgames.accessories != ''].str.split('|')

In [446]:
x['accessories_list'] = x.accessories[x.accessories != ''].str.split('|')

In [452]:
boardgames[['boardgame_id', 'accessories_list']][boardgames.accessories != ''].explode('accessories_list').drop_duplicates()


,boardgame_id,accessories_list
3,35462,Lock 'n Load: Noville – Bastogne's Outpost: X-...
8,36,Federation & Empire: Large Scale Map
13,1589,Deluxe Space Battle Maps
19,37061,Deluxe Space Battle Maps
20,35400,Nations At War: Companion Book
21,345931,The Shivers
21,345931,The Shivers (Deluxe Edition)
23,41047,Admin Test Accessory
23,41047,Test item expansion to accessory
31,29663,Deluxe Space Battle Maps


In [453]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
# for i in honors:
for index, row in boardgames[['boardgame_id', 'accessories_list']][boardgames.accessories != ''].explode('accessories_list').drop_duplicates().iterrows():
#     try:
    i = row.accessories_list
    if i != '':
        cursor.execute('''
            SELECT accessory_id
            FROM accessories
            WHERE accessory = ?
            ''', (i,))
        accessory_id = cursor.fetchall()[0][0]
        
        cursor.execute('''
            SELECT *
            FROM boardgame_accessories
            WHERE (boardgame_id = ?) & (accessory_id = ?)
            ''', (int(row.boardgame_id), accessory_id))
        flag = cursor.fetchall()
        if flag == []:
            cursor.execute('''
                INSERT INTO boardgame_accessories (boardgame_id, accessory_id) VALUES (?, ?)
                ''', (int(row.boardgame_id), accessory_id))

# commit the changes
conn.commit()

# close the connection
conn.close()

## Mechanics

In [289]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
for i in boardgames.mechanics.apply(lambda x: x.split('|')).explode().unique().tolist():
    if i != '':
        cursor.execute('''
            INSERT INTO mechanics (mechanic) VALUES (?)
            ''', (i,))

# commit the changes
conn.commit()

# close the connection
conn.close()

## Boardgame_mechanics

In [295]:
boardgames['mechanics_list'] = boardgames.mechanics.apply(lambda x: x.split('|'))

In [457]:
x['mechanics_list'] = x.mechanics.apply(lambda x: x.split('|'))

In [458]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
# for i in honors:
for index, row in boardgames[['boardgame_id', 'mechanics_list']][boardgames.mechanics != ''].explode('mechanics_list').iterrows():
#     try:
    i = row.mechanics_list
    if i != '':
        cursor.execute('''
            SELECT mechanic_id
            FROM mechanics
            WHERE mechanic = ?
            ''', (i,))
        mechanic_id = cursor.fetchall()[0][0]

        cursor.execute('''
            SELECT *
            FROM boardgame_mechanics
            WHERE (boardgame_id = ?) & (mechanic_id = ?)
            ''', (int(row.boardgame_id), mechanic_id))
        flag = cursor.fetchall()
        if flag == []:
            cursor.execute('''
                INSERT INTO boardgame_mechanics (boardgame_id, mechanic_id) VALUES (?, ?)
                ''', (int(row.boardgame_id), mechanic_id))

# commit the changes
conn.commit()

# close the connection
conn.close()

## Categories

In [309]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
for i in boardgames.category.apply(lambda x: x.split('|')).explode().unique().tolist()[:-1]:
    if i != '':
        cursor.execute('''
            INSERT INTO categories (category) VALUES (?)
            ''', (i,))

# commit the changes
conn.commit()

# close the connection
conn.close()

## Boardgame_categories

In [376]:
boardgames['category_list'] = boardgames.category.apply(lambda x: x.split('|'))

In [459]:
x['category_list'] = x.category.apply(lambda x: x.split('|'))

In [461]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
# for i in honors:
for index, row in boardgames[['boardgame_id', 'category_list']][boardgames.category != ''].explode('category_list').iterrows():
#     try:
    i = row.category_list
    if i == 'Print  Play':
        i = 'Print & Play'
    if i != '':
        cursor.execute('''
            SELECT category_id
            FROM categories
            WHERE category = ?
            ''', (i,))
        category_id = cursor.fetchall()[0][0]

        cursor.execute('''
            SELECT *
            FROM boardgame_categories
            WHERE (boardgame_id = ?) & (category_id = ?)
            ''', (int(row.boardgame_id), category_id))
        flag = cursor.fetchall()
        if flag == []:
            cursor.execute('''
                INSERT INTO boardgame_categories (boardgame_id, category_id) VALUES (?, ?)
                ''', (int(row.boardgame_id), category_id))

# commit the changes
conn.commit()

# close the connection
conn.close()

## Subdomains

In [480]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
for index, row in boardgames[boardgames.subdomains != ''].iterrows():  
    cursor.execute('''
        SELECT *
        FROM subdomains
        WHERE boardgame_id = ?
        ''', (int(row.boardgame_id), ))
    flag = cursor.fetchall()
    if flag == []:
        wargames = 'Wargames' in row.subdomains
        abstract = 'Abstract Games' in row.subdomains
        party = 'Party Games' in row.subdomains
        childrens = "Children's Games" in row.subdomains
        thematic = 'Thematic Games' in row.subdomains
        customizable = 'Customizable Games' in row.subdomains
        family = 'Family Games' in row.subdomains
        strategy =  'Strategy Games' in row.subdomains
        
        cursor.execute('''
            INSERT INTO subdomains (boardgame_id, wargames, abstract, party, childrens, thematic, customizable, family, strategy) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (int(row.boardgame_id), wargames, abstract, party, childrens, thematic, customizable, family, strategy))

# commit the changes
conn.commit()

# close the connection
conn.close()

# STOP HERE
Fill all data into next tables in our dataset:
- expansions
- boardgame_expansions
- persons
- boardgame_artists
- boardgame_designers
- boardgame_graphic_designers
- implementations
- podcast_episodes
- boardgame_podcast_episodes
- conditions
- sales
- boardgame_sales
- ranks

In [438]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()

# Execute the PRAGMA statement to get the table schema
# cursor.execute("PRAGMA table_info(ratings)")

# Fetch all the rows returned by the statement
# rows = cursor.fetchall()

# Print the table schema
# for row in rows:
#     print(row)

cursor.execute('''
        SELECT *
        FROM boardgame_categories
        ''')
data = cursor.fetchall()
# close the connection
conn.close()

len(data)

357196

## !!! Expansions

In [190]:
boardgames['list_expansions'] = boardgames.expansions[boardgames.expansions != ''].apply(lambda x: x.split('|'))
boardgames[['boardgame_id', 'title','list_expansions']][boardgames.expansions != ''].explode('list_expansions')

,boardgame_id,title,list_expansions
1,259048,Star Wars: X-Wing (Second Edition) – RZ-2 A-Wi...,Star Wars: X-Wing (Second Edition)
6,77423,The Lord of the Rings: The Card Game,The Lord of the Rings: The Card Game – A Journ...
6,77423,The Lord of the Rings: The Card Game,The Lord of the Rings: The Card Game – A Shado...
6,77423,The Lord of the Rings: The Card Game,The Lord of the Rings: The Card Game – A Storm...
6,77423,The Lord of the Rings: The Card Game,The Lord of the Rings: The Card Game – Across ...
...,...,...,...
134721,237950,The Shared Dream: Night Terrors,The Shared Dream
134722,164368,The Lord of the Rings: The Return of the King ...,The Lord of the Rings: The Return of the King ...
134730,210337,World Monuments: Monument Valley,World Monuments
134732,217458,XenoShyft: Forbidden Sciences,XenoShyft: Dreadmire


In [192]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()

# Execute the PRAGMA statement to get the table schema
# cursor.execute("PRAGMA table_info(ratings)")

# Fetch all the rows returned by the statement
# rows = cursor.fetchall()

# Print the table schema
# for row in rows:
#     print(row)
for_check_bg = []
for index, row in boardgames[['boardgame_id', 'title','list_expansions']][boardgames.expansions != ''].explode('list_expansions').iterrows():
# i = 'Star Wars: X-Wing (Second Edition)'
    try:
        cursor.execute('''
                SELECT boardgame_id, title
                FROM boardgames
                WHERE title = ?
                ''',(row.list_expansions,))
        data = cursor.fetchall()[0][0]
    except:
        for_check_bg.append(row.list_expansions)
# close the connection
conn.close()

In [193]:
len(for_check_bg)

7444

In [197]:
# pd.DataFrame(for_check_bg, columns=['expansion']).to_csv('../expansion.csv', index=False)

In [213]:
sleep(0.3)

In [314]:
%%time 
search_query = 'https://boardgamegeek.com/geeksearch.php?action=search&objecttype=boardgame&q='
expansions = pd.DataFrame()
for bg in for_check_bg:
    r = re.get(search_query + bg)
    soup = BeautifulSoup(r.text)
    rows = soup.find_all('tr', id='row_')
    for i in rows:
        link = i.find('a', class_='primary').get('href')
        bg_id = int(link.split("/")[2])
#         print(bg_id, bg, link)
        expansions = pd.concat([expansions, pd.DataFrame({'boardgame_id': bg_id, 'title': bg, 'link': link}.values(), index=[['boardgame_id', 'title', 'link']]).T], axis=0)
    sleep(0.3)
#     break

CPU times: user 4min 44s, sys: 18.4 s, total: 5min 2s
Wall time: 1h 18min 24s


In [366]:
# expansions.reset_index(drop=True).to_csv('../expansion_scpared.csv', index=False)
len(set(expansions.iloc[:,0].to_list()))

2866

In [364]:
ids_bg = []
for i in data:
    ids_bg.append(i[0])

In [370]:
len(set(expansions.iloc[:,0].to_list()) - set(ids_bg))

1653

In [437]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()

# Execute the PRAGMA statement to get the table schema
# cursor.execute("PRAGMA table_info(ratings)")

# Fetch all the rows returned by the statement
# rows = cursor.fetchall()

# Print the table schema
# for row in rows:
#     print(row)

cursor.execute('''
        SELECT *
        FROM boardgame_honors
        ''')
data = cursor.fetchall()
# close the connection
conn.close()

len(data)

14633

In [231]:
pd.DataFrame({'boardgame_id': 13123, 'title': '12312', 'link': '123142411link'}.values(), index=[['boardgame_id', 'title', 'link']]).T




,boardgame_id,title,link
0,13123,12312,123142411link


In [ ]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
for i in awards:
#     print(int(row.boardgame_id), row.description)
    cursor.execute('''
        INSERT INTO awards (award) VALUES (?)
        ''', (i,))

# commit the changes
# conn.commit()

# close the connection
conn.close()

In [436]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()

# Execute the PRAGMA statement to get the table schema
# cursor.execute("PRAGMA table_info(ratings)")

# Fetch all the rows returned by the statement
rows = cursor.fetchall()

# Print the table schema
for row in rows:
    print(row)

cursor.execute('''
        SELECT *
        FROM boardgame_publishers
        ''')
data = cursor.fetchall()
# close the connection
conn.close()

len(data)

204615

In [93]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

# Execute the PRAGMA statement to get the table schema
cursor.execute("PRAGMA table_info(ratings)")

# Fetch all the rows returned by the statement
rows = cursor.fetchall()

# Print the table schema
for row in rows:
    print(row)

cursor.execute('''
        SELECT *
        FROM ratings
        ''')
data = cursor.fetchall()
# close the connection
conn.close()

(0, 'rating_id', 'INTEGER', 0, None, 1)
(1, 'user_id', 'INTEGER', 1, None, 0)
(2, 'boardgame_id', 'INTEGER', 1, None, 0)
(3, 'rating', 'FLOAT', 1, None, 0)
(4, 'num_of_plays', 'INTEGER', 1, None, 0)
(5, 'comment_id', 'INTEGER', 0, None, 0)
(6, 'own', 'BOOLEAN', 1, '0', 0)
(7, 'prevowned', 'BOOLEAN', 1, '0', 0)
(8, 'for_trade', 'BOOLEAN', 1, '0', 0)
(9, 'want', 'BOOLEAN', 1, '0', 0)
(10, 'want_to_play', 'BOOLEAN', 1, '0', 0)
(11, 'want_to_buy', 'BOOLEAN', 1, '0', 0)
(12, 'wishlist', 'BOOLEAN', 1, '0', 0)
(13, 'preordered', 'BOOLEAN', 1, '0', 0)
(14, 'last_modified', 'DATE', 1, None, 0)


In [ ]:
cursor.execute('''
        CREATE TABLE ratings (
        rating_id INTEGER PRIMARY KEY 
        user_id INTEGER NOT NULL REFERENCES users(user_id),
        boardgame_id INTEGER NOT NULL REFERENCES boardgames(boardgame_id),
        rating FLOAT NOT NULL,
        num_of_plays INTEGER NOT NULL,
        comment_id INTEGER REFERENCES comments(comment_id),
        own BOOLEAN DEFAULT 0 NOT NULL,
        prevowned BOOLEAN DEFAULT 0 NOT NULL,
        for_trade BOOLEAN DEFAULT 0 NOT NULL,
        want BOOLEAN DEFAULT 0 NOT NULL,
        want_to_play BOOLEAN DEFAULT 0 NOT NULL,
        want_to_buy BOOLEAN DEFAULT 0 NOT NULL,
        wishlist BOOLEAN DEFAULT 0 NOT NULL,
        preordered BOOLEAN DEFAULT 0 NOT NULL,
        last_modified DATE NOT NULL
        );
''')

In [429]:
# connect to the database
conn = sqlite3.connect(PATH_TO_DB)

# create a cursor object
cursor = conn.cursor()

cursor.execute("DROP TABLE boardgame_honors")
cursor.execute('''
        CREATE TABLE boardgame_honors (
        boardgame_id INTEGER REFERENCES boardgames(boardgame_id),
        honor_id INTEGER REFERENCES honors(honor_id),
        PRIMARY KEY (boardgame_id, honor_id)
        )
''')

# data = cursor.fetchall()
# close the connection
conn.close()

In [36]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()

for index, row in tqdm(x.iterrows()):
    # execute the insert statements
    if row.image_link != '':
        cursor.execute('SELECT link_id FROM links WHERE link = ?', (row.image_link,))
        link_id = cursor.fetchall()[0][0]
        cursor.execute("UPDATE boardgames SET image_link_id=? WHERE boardgame_id=?", (link_id, int(row.boardgame_id)))
        
#         print(link_id)
#     break
# commit the changes
# conn.commit()

# close the connection
conn.close()

0it [00:00, ?it/s]

In [796]:
# connect to the database
conn = sqlite3.connect('../Board-Game-Data/board_game_database.db')

# create a cursor object
cursor = conn.cursor()
# execute the insert statements
for i in data:
    foj
    cursor.execute('''
        INSERT INTO users (user_id, nickname, country_id, website_id, registration_date, last_profile_update, last_login, last_check)
        ''', i)
data = cursor.fetchall()


# close the connection
conn.close()

OperationalError: incomplete input

In [ ]:
user_id INTEGER PRIMARY KEY,
        nickname TEXT NOT NULL UNIQUE,
        country_id INTEGER REFERENCES сountries(country_id),
        website_id INTEGER REFERENCES links(link_id),
        registration_date DATE,
        last_profile_update DATE,
        last_login DATE,
        last_check

In [79]:
get_data_from_db(f"""
                SELECT *
                FROM users 
                """, out=True)

[(1, ' me ', 1, None, None, '2023-02-01')]

In [74]:
for index, row in ratings.iterrows():
    print(row)
    user_id = get_data_from_db(f"""
                SELECT user_id
                FROM users 
                WHERE nickname = ?
                """, (row.nickname,), out=True)
    print(user_id)
    boardgame_id = int(row.boardgame_id)
    rating = float(row.rating)
    num_of_plays = int(row.num_of_plays)
    comment_id = get_data_from_db(f"""
                SELECT comment_id
                FROM comments 
                WHERE comment = ?
                """, (row.comment,), out=True)
    print(comment_id)
    own = int(row.own)
    prevowned = int(row.prevowned)
    for_trade = int(row.for_trade)
    want = int(row.want)
    want_to_play = int(row.want_to_play)
    want_to_buy = int(row.want_to_buy)
    wishlist = int(row.wishlist)
    preordered = int(row.preordered)
    last_modified = row.last_modified[:10]
    print(last_modified)
    break

nickname                    DiVo1975
title                      Anachrony
boardgame_id                  185343
rating                           9.0
num_of_plays                       2
comment                         None
own                                0
prevowned                          1
fortrade                           0
want                               0
wanttoplay                         0
wanttobuy                          0
wishlist                           0
preordered                         0
last_modified    2022-11-06 14:02:30
Name: 0, dtype: object
[]
[]


AttributeError: 'Series' object has no attribute 'for_trade'

In [399]:
for index, row in boadrgames[boadrgames.year_published != ""].iterrows():
    if row.year_published != '0': # year_published
        year_published = int(row.year_published)
    else:
        year_published = np.nan
        
    if row.minplayers != '0': # min_players
        min_players = int(row.minplayers)
    else:
        min_players = np.nan
        
    if int(row.maxplayers) < min_players: # max_players
        max_players = np.nan
    elif row.maxplayers == '0':
        max_players = min_players
    else:
        max_players = int(row.maxplayers)
        
    if row.minplaytime != '0': # min_playtime
        min_playtime = int(row.minplaytime)
    else:
        min_playtime = np.nan
        
    if int(row.maxplaytime) < min_playtime: # max_playtime
        max_playtime = np.nan
    elif row.maxplaytime == '0':
        max_playtime = min_playtime
    else:
        max_playtime = int(row.maxplaytime)
        
    if (row.age != '0') & (row.age != ''): # age
        age = int(row.age)
    else:
        age = np.nan
        
    if row.users_rated != '': # users_rated
        users_rated = int(row.users_rated)
    else:
        users_rated = np.nan
        
    if row.average_rating != '': # average_rating
        average_rating = float(row.average_rating)
    else:
        average_rating = np.nan
        
    if row.bayes_average_rating != '': # bayes_average_rating
        bayes_average_rating = float(row.bayes_average_rating)
    else:
        bayes_average_rating = np.nan
    
    if row.stddev != '': # stddev
        std_dev = float(row.stddev)
    else:
        std_dev = np.nan
    
    if row.owned != '': # owned
        owned = int(row.owned)
    else:
        owned = np.nan 
        
    if row.trading != '': # trading
        trading = int(row.trading)
    else:
        trading = np.nan
    
    if row.wishing != '': # wishing
        wishing = int(row.wishing)
    else:
        wishing = np.nan 
        
    if row.num_of_comments != '': # num_of_comments
        num_of_comments = int(row.num_of_comments)
    else:
        num_of_comments = np.nan
        
    if row.num_of_weights != '': # num_of_weights
        num_of_weights = int(row.num_of_weights)
    else:
        num_of_weights = np.nan
        
    if row.average_weight != '': # average_weight
        average_weight = float(row.average_weight)
    else:
        average_weight = np.nan
        
    if (isNaN(row.main_publisher)) | (row.main_publisher == ''): # main_publisher_id
        main_publisher_id = np.nan
    else:
        main_publisher_id = get_data_from_db(f"""
                SELECT publisher_id
                FROM publishers 
                WHERE publisher = '{row.main_publisher}'
                """)[0][0]
        
    if (isNaN(row.thumbnail_link)) | (row.thumbnail_link == ''): # thumbnail_link_id
        thumbnail_link_id = np.nan
        
    else:
        thumbnail_link_id = get_data_from_db(f"""
                SELECT link_id
                FROM links 
                WHERE link = '{row.thumbnail_link}'
                """)[0][0]
        
    if (isNaN(row.image_link)) | (row.image_link == ''): # image_link_id
        image_link_id = np.nan
        
    else:
        image_link_id = get_data_from_db(f"""
                SELECT link_id
                FROM links 
                WHERE link = '{row.image_link}'
                """)[0][0]

    print(len((int(row.boardgame_id), row.title, year_published, min_players, max_players, 
           min_playtime, max_playtime, age, users_rated, average_rating, bayes_average_rating, std_dev,
           owned, trading, wishing, num_of_comments, 
           num_of_weights, average_weight, main_publisher_id, thumbnail_link_id, image_link_id)))
    break

        

21


In [392]:
get_data_from_db(f"""
                SELECT publisher_id
                FROM linkes 
                WHERE publisher = '{row.main_publisher}'
                """)[0][0]

5688

In [393]:
boadrgames[(boadrgames.year_published != "") & (boadrgames.thumbnail_link == "")].iloc[:,19:]



,ranks,main_publisher,description,publishers,honors,expansions,accessories,artists,mechanics,category,...,graphic_designers,subdomains,implementations,suggested_numplayers,podcast_episodes,comments,marketplace_history,marketplace_listings,thumbnail_link,image_link
17,Board Game Rank|Not Ranked||Abstract Game Rank...,(Self-Published),"In this card and dice-free horse racing game, ...",(Self-Published),,,,,Simulation,Abstract Strategy|Animals|Racing,...,,Abstract Games,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,Coudloc|Played once,,,,
24,Board Game Rank|Not Ranked,(Web published),"2 player traditional card game, combining tric...",(Web published),,,,,Melding and Splaying|Memory|Set Collection|Tri...,Card Game,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,
54,Board Game Rank|Not Ranked,(Self-Published),"Overview<br/>Explore a game of Science, Techno...",(Self-Published),,,,,Moving Multiple Units|Pieces as Map|Real-Time|...,Civilization|Educational|Fantasy|Novel-based|R...,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,"bmcole|As the game designer, I find it only fi...",,,,
58,Board Game Rank|Not Ranked,Fitzpatrick Bros.,&quot;An exciting radio 'quiz' game for all th...,Fitzpatrick Bros.,,,,,,Card Game|Children's Game|Movies / TV / Radio ...,...,,,Meet the Missus,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,
68,Board Game Rank|Not Ranked,D. Carvalho,"A race game based on Paul Pry (1825), a farcic...",D. Carvalho,,,,,Roll / Spin and Move,Dice,...,,,,0+||Best~0|Recommended~0|Not Recommended~0,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134640,Board Game Rank|Not Ranked,(Self-Published),Simple roll and move game based on characters ...,(Self-Published),,,,Ulf Grimås,Point to Point Movement|Roll / Spin and Move,Children's Game,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,
134680,Board Game Rank|Not Ranked,(Self-Published),"Just like the name suggests, The Art of War pa...",(Self-Published),,,,,Area Movement|Deck Construction|Dice Rolling|G...,Civil War|Dice|Economic|Fantasy|Math|Medieval|...,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,
134685,Board Game Rank|Not Ranked,(Self-Published),"Rock, Roadies and Roll is a game for 3 to 5 pl...",(Self-Published),,,,,Auction/Bidding|Worker Placement,Music,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,Man or Astroman|Summer 2012: First playtest co...,,,,
134697,Board Game Rank|Not Ranked,Fat Brain Toy Company,It&rsquo;s a silly sequencing memory game! Kid...,Fat Brain Toy Company,2017/Fall Parents' Choice Silver Honor Winner,,,,Memory,Animals|Card Game|Children's Game|Humor|Memory...,...,,,,1||Best~0|Recommended~0|Not Recommended~0|||2|...,,,,,,


In [311]:
boadrgames.year_published.unique()

array(['2002', '2018', '2003', '2017', '2020', '2011', '2007', '1975',
       '2005', '2012', '2015', '1987', '2016', '2010', '1989', '2008',
       '2019', '2009', '2006', '0', '2021', '1991', '1976', '1979',
       '2022', '1995', '2014', '1992', '2023', '1940', '1998', '1977',
       '1811', '1984', '1988', '2013', '1983', '1999', '1970', '1967',
       '1985', '2001', '1969', '1993', '1937', '2004', '1996', '2000',
       '1990', '1986', '1994', '1809', '1962', '1973', '1981', '1953',
       '1972', '1997', '1978', '1897', '1951', '1980', '1947', '1964',
       '1959', '1961', '1925', '1700', '1915', '1960', '1954', '1974',
       '1982', '1966', '1888', '1930', '1936', '1968', '1926', '1945',
       '1903', '1921', '1971', '1900', '1938', '1957', '1932', '1935',
       '1950', '1928', '2024', '1810', '1955', '1890', '1956', '1958',
       '1949', '1963', '1863', '1701', '1948', '1870', '1965', '1913',
       '1791', '1898', '1931', '', '1908', '1896', '1923', '1922', '1910',
     

In [256]:
get_data_from_db("""
                SELECT city_id
                FROM cities 
                WHERE city = 'London'
                """)

[(6,)]

Order of adding info into database:
- country +
- state +
- city +
- users +

- links +
- publishers +
- boardgames +

- comments +
- ratings +

Other boardgame data
- descriptions
- boardgame_publishers
- honors
- boardgame_honors
- expansions
- boardgame_expansions
- accessories
- boardgame_accessories
- persons
- boardgame_artists
- mechanics
- boardgame_mechanics
- categories
- boardgame_categories
- boardgame_designers
- boardgame_graphic_designers
- subdomains
- implementations
- podcast_episodes
- boardgame_podcast_episodes
- sales
- boardgame_sales
- ranks

In [140]:
list(boardgames.columns)

['boardgame_id',
 'title',
 'year_published',
 'minplayers',
 'maxplayers',
 'minplaytime',
 'maxplaytime',
 'age',
 'users_rated',
 'average_rating',
 'bayes_average_rating',
 'median',
 'stddev',
 'owned',
 'trading',
 'wishing',
 'num_of_comments',
 'num_of_weights',
 'average_weight',
 'ranks',
 'main_publisher',
 'description',
 'publishers',
 'honors',
 'expansions',
 'accessories',
 'artists',
 'mechanics',
 'category',
 'designers',
 'graphic_designers',
 'subdomains',
 'implementations',
 'suggested_numplayers',
 'podcast_episodes',
 'comments',
 'marketplace_history',
 'marketplace_listings',
 'thumbnail_link',
 'image_link']

In [141]:
boardgames.honors[boardgames.honors != ""]

6         2011 Golden Geek Best 2-Player Board Game Nomi...
41        2018 Hungarian Board Game Award Nominee|2018 H...
99                        1987 Spiel des Jahres Recommended
122       2021 5 Seasons Best International Party&Coop N...
170             2011/Fall Parents' Choice Gold Award Winner
                                ...                        
134628     2020 UK Games Expo Best Miniatures Rules Nominee
134649    2019 5 Seasons Best Portuguese Expansion Nomin...
134669    2011/Spring Parents' Choice Approved Award Winner
134697        2017/Fall Parents' Choice Silver Honor Winner
134707    1987 Charles S. Roberts Best Pre-World War II ...
Name: honors, Length: 6041, dtype: object

In [138]:
boardgames[boardgames.subdomains != ''].iloc[:,27]

6         Cooperative Game|Deck Construction|Events|Hand...
7                                              Dice Rolling
15                                            Modular Board
17                                               Simulation
19                   Memory|Pattern Building|Tile Placement
                                ...                        
134712                                                     
134713    Hand Management|Highest-Lowest Scoring|Pattern...
134718    Network and Route Building|Simulation|Stock Ho...
134733                          Hand Management|Negotiation
134737                                        Modular Board
Name: mechanics, Length: 28288, dtype: object

In [96]:
set(boardgames.subdomains[boardgames.subdomains != ''])

{'Abstract Games',
 "Abstract Games|Children's Games",
 "Abstract Games|Children's Games|Wargames",
 'Abstract Games|Customizable Games',
 'Abstract Games|Family Games',
 'Abstract Games|Party Games',
 'Abstract Games|Strategy Games',
 'Abstract Games|Thematic Games',
 'Abstract Games|Wargames',
 "Children's Games",
 "Children's Games|Customizable Games",
 "Children's Games|Family Games",
 "Children's Games|Party Games",
 "Children's Games|Strategy Games",
 "Children's Games|Thematic Games",
 "Children's Games|Wargames",
 'Customizable Games',
 'Customizable Games|Family Games',
 'Customizable Games|Strategy Games',
 'Customizable Games|Thematic Games',
 'Customizable Games|Wargames',
 'Family Games',
 'Family Games|Party Games',
 'Family Games|Strategy Games',
 'Family Games|Strategy Games|Thematic Games',
 'Family Games|Thematic Games',
 'Family Games|Wargames',
 'Party Games',
 'Party Games|Strategy Games',
 'Party Games|Thematic Games',
 'Party Games|Wargames',
 'Strategy Games',
 

In [102]:
def ranks(list_ranks):
    ranks = []
    for i in list_ranks:
        for j in i.split('||'):
            ranks.append(j.split('|')[0])
    return list(set(ranks))

ranks(list(boardgames.ranks[boardgames.ranks != '']))

['Customizable Rank',
 'Atari ST Rank',
 'RPG Item Rank',
 'Arcade Rank',
 'Strategy Game Rank',
 'Board Game Rank',
 "Children's Game Rank",
 'Family Game Rank',
 'Abstract Game Rank',
 'Board Game Issue Rank',
 'Party Game Rank',
 'Thematic Rank',
 'Commodore 64 Rank',
 'Accessory Rank',
 'Amiga Rank',
 'War Game Rank',
 'Video Game Rank']

In [128]:
def categories(list_categories):
    categories = []
    for i in list_categories:
        categories.append(i.split('|')[0])
    return list(set(categories))

categories(list(boardgames.category[boardgames.category != '']))

['Game System',
 'American Revolutionary War',
 'Aviation / Flight',
 'Trains',
 'Real-time',
 'Abstract Strategy',
 'Industry / Manufacturing',
 'Bluffing',
 'Nautical',
 'Mafia',
 'Zombies',
 'Wargame',
 'Expansion for Base-game',
 'Mythology',
 'Print  Play',
 'Video Game Theme',
 'Medieval',
 'Age of Reason',
 'Miniatures',
 'American Indian Wars',
 'Party Game',
 'Trivia',
 'Political',
 'Modern Warfare',
 'Fantasy',
 'Ancient',
 'Farming',
 'Environmental',
 'Deduction',
 'Animals',
 'Comic Book / Strip',
 'Prehistoric',
 'Mature / Adult',
 'Racing',
 'Civil War',
 'Print & Play',
 'Card Game',
 'Exploration',
 'Medical',
 'Renaissance',
 'Dice',
 'Fighting',
 'Humor',
 'World War II',
 'Math',
 'Napoleonic',
 'Book',
 'Pirates',
 'Educational',
 'Music',
 'Novel-based',
 'Word Game',
 'Puzzle',
 'Maze',
 'Electronic',
 'Pike and Shot',
 'Arabian',
 'Memory',
 'Sports',
 'Post-Napoleonic',
 'World War I',
 'Number',
 'Space Exploration',
 'Horror',
 'City Building',
 'Territory B

In [97]:
categories(list(set(boardgames.subdomains[boardgames.subdomains != ''])))

['Wargames',
 'Abstract Games',
 'Party Games',
 "Children's Games",
 'Thematic Games',
 'Customizable Games',
 'Family Games',
 'Strategy Games']

In [99]:
len(categories(list(set(boardgames.graphic_designers[boardgames.graphic_designers != ''])))) 

1623

In [103]:
len(categories(list(set(boardgames.designers[boardgames.designers != '']))))

27351

In [142]:
len(categories(list(set(boardgames.honors[boardgames.honors != ''])))) 

2306

In [146]:
years = []
award = []
for i in categories(list(set(boardgames.honors[boardgames.honors != '']))):
    years.append(int(i[:4]))
    award.append(str(i[5:]))
#     print(int(i[:4]), '---', str(i[5:]))

2019 --- Gioco dell’Anno Nominee
2001 --- Essener Feder Best Written Rules Winner
1998 --- Spiel des Jahres Recommended
2001 --- Charles S. Roberts Best Wargame Graphics Nominee
2017 --- Guldbrikken Best Children's Game Nominee
2018 --- Origins Awards Best Board Game Nominee
2017 --- Spiel des Jahres Recommended
2002 --- Fairplay À la carte Runner-up
1997 --- Fairplay À la carte Runner-up
2015 --- Årets Spill Best Family Game Nominee
1986 --- Gra Roku Game of the Year Winner
2018 --- Graf Ludo Best Family Game Graphics Winner
2010 --- Diana Jones Award for Excellence in Gaming Nominee
1995 --- Årets Spil Best Children's Game Winner
2020 --- UK Games Expo Best Miniatures Rules Judges Award Winner
1979 --- Spiel des Jahres "Beautiful Game" Winner
2018 --- RPC Fantasy Award Board Game Winner
2007 --- Lucca Game Best Card Game
2013 --- Golden Geek Best Board Game Expansion Nominee
2021 --- UK Games Expo Best Variant or Expansion Judges Award Winner
2019 --- Board Game Quest Awards Best Fam

In [170]:
def history(x):
    for i in x.split('|||'):
        print(i)
        break
history(boadrgames.marketplace_history[boadrgames.marketplace_history != ''].iloc[2])

unix_saledate|1301347058||saletime|120837||price|25.95||currency|USD||condition|new


In [ ]:
boadrgames.marketplace_history[boadrgames.marketplace_history != ''].iloc[2]

In [112]:
len(set(ranks(list(boardgames.comments[boardgames.comments != '']))) - set(users.nickname))

34222

In [158]:
len(split_once(list(boardgames[boardgames.accessories != ''].accessories)))

6794

In [ ]:
# # users
# for ind in tqdm(range(len(users))):
#     username = users.iloc[ind]["nickname"]
#     country = users.iloc[ind]["country"]
#     state = users.iloc[ind]["state"]
#     city = users.iloc[ind]["city"]
#     link = users.iloc[ind]["link"]
#     query = (f"""
#     UPDATE users
#     SET WorkComment = '1'
#     WHERE ID='{id_excel}'
#    """)
# #ниже отправляю по одному в бд
#     conn.execute(query)
# conn.close()
# eng.dispose()